### This guide will show how to host a hugging face LLM in Union serverless actors environment. 

<a target="_blank" href="https://colab.research.google.com/github/unionai-oss/phi3-on-union-actors/blob/main/tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Sign up for Union serveless (free):
https://www.union.ai/

It can take upto 5 minutes to get your on boarded email. If you do not get approved reach out on the Flyte community Slack. 

Once you get your confirmation email you should be able to verify access at visiting serverless.union.ai

See the full repo here: https://github.com/unionai-oss/phi3-on-union-actors

In [ ]:

# install the requirements if you haven't already
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/unionai-oss/phi3-on-union-actors
    %cd phi3-on-union-actors
    !pip install -r requirements.txt

Autheticate this device with your Union account.

In [45]:
!union create login --serverless --auth device-flow

Login successful into serverless
E0000 00:00:1737684853.509548 28073574 init.cc:232] grpc_wait_for_shutdown_with_timeout() timed out.


Lets run a workflow that uses the phi3 instruct LLM model from Hugging Face to generate text.

See the code we're running in [asks/phi3.py](tasks/phi3.py) and the workflow in [workflows/phi3_workflow.py](workflows/phi3_response_wf.py)

If first time runninng this workflow the container image will have to build which may take a few mins. And the first time an actor workflow is run it will spin up a container taking a few minutes as well. Once it's up you'll be able to call it until time-to-life occurs.

When workflow is running you'll get a generated link to view it's progress in the Union UI.



In [ ]:
!union register workflows/model_infernce.py 

Lets call our LLM model to generate text with the Union remote API. 

This allows us to run the workflow from other applications or scripts.


In [46]:
from union.remote import UnionRemote
# Create a remote connection
remote = UnionRemote()

18:14:20.181248 WARNING  remote.py:286 - Jupyter notebook and interactive task  
                         support is still alpha.

In [47]:
def predict_with_actor(query):

    inputs = {"query": query,}

    workflow = remote.fetch_workflow(name="workflows.model_infernce.wf_text_gen")
    execution = remote.execute(workflow, inputs=inputs, wait=True) # wait=True will block until the execution is complete

    # print(execution.outputs)

    return execution.outputs['o0']

If you're calling this after the initial TTL expored on your container you'll have to wait for the container to spin up again. You can adjust the TTL for `ActorEnvironment` in the container file [containers.py](containers.py)

In [63]:
print(predict_with_actor("What is a bee?"))

 A bee is a flying insect that is known for its role in pollination. Bees are a part of the Hymenoptera order and are classified under the family Apidae. They are characterized by their hairy bodies, two pairs of wings, and a stinger. Bees are social insects and live in colonies, with a queen bee, worker bees, and drones. They are known for producing honey and beeswax. Bees play a crucial role in the ecosystem by pollinating flowers, which helps in the reproduction of plants. They are also important for agriculture as they help in the production of fruits, vegetables, and nuts.


In [64]:
print(predict_with_actor("What is a fly?"))

 A fly is a small, winged insect belonging to the order Diptera. They are characterized by their two wings, compound eyes, and a pair of antennae. Flies are found in almost every habitat on Earth and are known for their ability to fly quickly and maneuver in tight spaces. They play important roles in ecosystems as pollinators and as a food source for other animals. However, some species of flies can also be pests, as they can spread diseases and contaminate food.


In [65]:
print(predict_with_actor("What is a ant?"))

 An ant is a small, social insect that belongs to the family Formicidae. They are known for their complex social structure and ability to work together to build intricate nests and forage for food. Ants have six legs, a segmented body, and antennae. They are found in almost every habitat on Earth, from rainforests to deserts. Ants are omnivores and feed on a variety of foods, including seeds, nectar, and other insects. They are also known for their ability to carry objects that are many times their own body weight.


In [66]:
print(predict_with_actor("What is a mantis?"))

 A mantis is a type of insect that belongs to the order Mantodea. They are known for their unique appearance, with elongated bodies, large compound eyes, and raptorial forelegs that they use to catch prey. Mantises are carnivorous and primarily feed on insects, but some larger species may also eat small vertebrates. They are found in various habitats around the world, including forests, grasslands, and even urban areas. Mantises are also known for their mating behavior, where the male will often mount the female and hold her still with his forelegs while mating.


If you want to host the gradio app on Hugging Face Spaces you can create union API key:


In [ ]:
!union create api-key admin --name gradio-hf-app